In [ ]:
# 浮动行权价格亚式期权相当于用有效期内的平均价格代期权行权价格的普通期权
def get_float_asian_price(s0, r, q, sigma, T, option_type):
    """
    定义函数计算亚式期权的价值
    param s0: 标的资产当前价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param option_type: 'call'或'put'
    return 返回期权权益金
    """
    N_simulations = 1000
    n_steps = 200
    dt = T / n_steps
    
    opt_sum = 0
    
    # 模拟 N_simulations 条价格路径：
    for i in range(N_simulations):
        s = s0           # 每次模拟标的资产价格都从当前价格开始演化                                     
        price_sum = 0

        # 模拟标的资产价格的几何布朗运动，确定不同价格路径下障碍条件是否会触及
        for j in range(n_steps):
            z = gauss(0,1)
            # 资产价格的几何布朗运动
            s *= e ** ((r - 0.5*(sigma**2))*dt + sigma*sqrt(dt)*z)
            
            # 记录价格路径中的价格极值
            price_sum += s
        
        price_avg = price_sum / n_steps
        
        # 计算一条价格路径对应的亚式期权的价值
        opt_sum += get_vanilla_price(s0, price_avg, r, q, sigma, T, option_type)
    
    return opt_sum/N_simulations    # 对所有价格路径下得到的期权价值取均值即附加障碍条件以后的期权价值

get_float_asian_price(s0, r, q, sigma, T, 'p')

# 固定行权价格亚式期权相当于用有效期内的平均价格替代标的资产终值计算收益的期权
def get_fixed_asian_price(s0, X, r, q, sigma, T, option_type):
    """
    定义函数计算回望期权的价值
    param s0: 标的资产当前价格
    param X: 期权行权价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param option_type: 'call'或'put'
    return 返回期权权益金
    """
    N_simulations = 1000
    n_steps = 200
    dt = T / n_steps
    
    value_sum = 0
    
    # 模拟 N_simulations 条价格路径：
    for i in range(N_simulations):
        s = s0           # 每次模拟标的资产价格都从当前价格开始演化                                     
        price_sum = 0

        # 模拟标的资产价格的几何布朗运动，确定不同价格路径下障碍条件是否会触及
        for j in range(n_steps):
            z = gauss(0,1)
            # 资产价格的几何布朗运动
            s *= e ** ((r - 0.5*(sigma**2))*dt + sigma*sqrt(dt)*z)
            # 记录价格路径中的价格极值
            price_sum += s
        
        price_avg = price_sum / n_steps
        
        # 计算一条价格路径对应的亚式浮动期权的价值
        if option_type.lower()[0] == 'c':
            value_sum += max((price_avg - X), 0)
        else:
            value_sum += max((X - price_avg), 0)
    
    return (e **(- r * T)) * value_sum / N_simulations    # 对所有价格路径下得到的期权价值取均值即附加障碍条件以后的期权价值

get_fixed_asian_price(s0, strike, r, q, sigma, T, 'p')